### Carregando o modelo

In [ ]:
!pip install tensorflow

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import nltk
import re
import json
import string
import spacy
import tensorflow as tf

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, LSTM, Conv1D, MaxPool1D
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score, precision_score, recall_score, f1_score

In [2]:
import joblib
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

# Viabilizadores
import joblib
from sklearn.model_selection import cross_val_predict, train_test_split
from sklearn.utils import shuffle
from sklearn.pipeline import make_pipeline

# Feature selection & relatório de resultados
from sklearn.feature_selection import SelectKBest, mutual_info_classif
from sklearn.metrics import classification_report , confusion_matrix, accuracy_score

In [3]:
from keras.models import load_model


In [37]:
def loadModel():
    name = input('escreva o nome do modelo a ser carregado:')
    savedModel = load_model(name)
    with open('wordIndex.json') as f:
        dicto = json.load(f)
    savedModel.summary()
    return savedModel, dicto

def loadTraditional():
    name = input('escreva o nome do modelo a ser carregado:')
    savedModel = joblib.load(name)
    print("model loaded")
    return savedModel

In [88]:
model=loadTraditional()

escreva o nome do modelo a ser carregado:SVC_7200_news_certo.pkl
model loaded


In [ ]:
tokenizer = Tokenizer()
model, tokenizer.word_index = loadModel()

### Notícias analisadas - True

In [6]:
true=[]
for i in range(1,54):
    try:
        with open(r'C:\Users\vduchi01\Downloads\tcc-fake-main\tcc-fake-main\3.noticias_true_maio22\\'+str(i)+'.txt', encoding ="utf8") as t:
            true.append(t.read())
    except:
        pass

In [47]:
len(true) # Ainda não truncado

53

In [48]:
labels = [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1]
len(labels)

53

In [50]:
labels_2 = ['true' if item == 0 else 'fake' for item in labels]
labels_2

['true',
 'true',
 'true',
 'true',
 'true',
 'true',
 'true',
 'true',
 'true',
 'true',
 'true',
 'true',
 'true',
 'true',
 'true',
 'true',
 'true',
 'true',
 'true',
 'true',
 'true',
 'true',
 'true',
 'true',
 'true',
 'true',
 'true',
 'true',
 'true',
 'true',
 'true',
 'true',
 'true',
 'true',
 'true',
 'true',
 'true',
 'true',
 'true',
 'true',
 'true',
 'true',
 'true',
 'true',
 'true',
 'true',
 'true',
 'true',
 'true',
 'true',
 'fake',
 'fake',
 'fake']

In [11]:
 df = pd.DataFrame(list(zip(true,labels)),columns=['true','labels'])

### Preprocessing

In [12]:
# Carregando o pacote de língua portuguesa para o processador Spacy
nlp = spacy.load('pt_core_news_sm')

In [13]:
# Defininido funções de preprocessamento

def removePunct(text):
    '''
    Removes any punctuation included in string.punctuation.
    '''
    translator = text.maketrans({key:'' for key in string.punctuation+'“”'}) # Translates any punctuation into ''
    return text.translate(translator)

def removeNumbers(text):
    '''
    Removes any number character in text.
    '''
    return re.sub('[0-9]', '' , text) # Translates any number into ''

def removeStopWords(string):
    '''
    Removes any portuguese stopwords, using Spacy's standard package.
    '''
    doc = nlp(string)
    return ' '.join([token.text for token in doc if token.is_stop is False])

def lemmatize(string):
    '''
    Lemmatizes text word-by-word. Notice that lemmatizing is not as harsh as stemming, which makes the final text easier to read and understand in common language.
    '''
    doc = nlp(string)
    return ' '.join([token.lemma_ for token in doc])

def prep(string, useStopWords = True, lemma = True):
    '''
    Executes previously defined preprocessing in text.
    '''

    result = removeNumbers(removePunct(string)).lower()
    
    if useStopWords and lemma:
        doc = nlp(result)
        result = ' '.join([token.lemma_ for token in doc if token.is_stop is False])
    elif useStopWords:
        doc = nlp(result)
        result = ' '.join([token.text for token in doc if token.is_stop is False])
    elif lemma:
        doc = nlp(result)
        result = ' '.join([token.lemma_ for token in doc])

    result = result.replace('\n',"")
    
    return result

In [51]:
sentence_t = df['true'].apply(prep)

In [54]:
len(sentence_t[1].split())

217

In [16]:
sentence_t.head()

0    presidente jair bolsonaro dizer caso avancar d...
1    comissão constituição Justiça Câmara ccj adiar...
2    deputado anunciar mesmo sessão acordo firmar A...
3    comissão constituição Justiça ccj senado adiar...
4    centro contingência coronavírus Paulo voltar r...
Name: true, dtype: object

### Classificação Trues

In [55]:
lista_t = sentence_t.to_list()

In [56]:
lista_t[1]

'comissão constituição Justiça Câmara ccj adiar terçafeira   votação requerimento convocação ministro Justiça Anderson Torres explique assassinato genivaldo jesus santo abordagem polícia Rodoviária federal prf Sergipe  presidente colegiado Arthur Maia uniãoba retirar proposta ofício autor matéria deputado delegado waldir uniãogo comparecer motivo saúde  regimento casa exigir presença autor proposta ser votar deputado afirmar tradição adotar ccj – caber autor encaminhar defesa votação requerimento  delegar waldir afirmar tv globo recuperar Covid aceitar pressão governo matéria ser votar parlamentar acreditar tema ser votar semana  imprescindível sociedade brasileiro ministro ir ccj acontecer dizer delegado waldir  praxe governo articule pedido convocação – presença autoridade obrigatório data hora marcar – ser converter convite – presença facultativo agendamento negociar autoridade  caso convocação ser aprovar anderson Torres ser comparecer justificar ausência contrário responder crime 

In [59]:
len(lista_t)

53

In [ ]:
maxlen = 100
sequences_t = tokenizer.texts_to_sequences(sentence_t)
padded_t = pad_sequences(sequences_t, maxlen=maxlen, padding='post',truncating='pre')

In [ ]:
len(padded_t[0])

In [ ]:
for i in range(0,50):
    print(len(sequences_t[i]))

Rodar no Tradicional

In [ ]:
lista_2 = [item.split() for item in lista_t]

In [ ]:
lista_2

In [ ]:
lista_3 = [noticia[0:100] for noticia in lista_2] #if len(noticia)>100]

In [ ]:
len(lista_3)

In [ ]:
for i in range (0,53):
    print(len(lista_3[i]))
    

In [ ]:
flat_list = [x for xs in lista_3 for x in xs]

In [ ]:
flat_list

In [ ]:
lista_4=[]
for i in lista_3:
    lista_4.append(' '.join(i))

In [ ]:
lista_4

In [57]:
def removeMinFreq(data, features, min_freq = 1):
    # Realiza a soma da frequência de cada palavra da matriz vetorizada
    cols_sum = np.sum(data, axis=0)

    del_indexes = []
    # Analisa cada valor das contagens de frequência do cols_sum, assignando um índice i, e salva o índice em del_indexes[]
    # quando a contagem for acima da min_freq
    for i, val in zip(range(len(cols_sum)), cols_sum):
        if val < min_freq:
            del_indexes.append(i)
            
    data = np.delete(data,del_indexes,1) # Deleta coluna
    features = np.delete(features,del_indexes,0) # Deleta linha
    return (data, features)


def normalizeData(data):
    rows_sum = np.sum(data, axis=1)
    data = (data.T / rows_sum).T
    return data

def loadCount(texts, min_freq = 1, binary = False, normalize = True):

    # Instanciando o CountVectorizer
    vectorizer = CountVectorizer(input = 'content', preprocessor = prep, encoding='utf-8', binary = binary);
    
    # Aplicando processo de vetorização
    data = np.array(vectorizer.fit_transform(lista_t).todense());
    features = np.array(vectorizer.get_feature_names())
    
    # Se min_freq for 1, então todos os tokens são considerados
    if(min_freq > 1):
        data, features = removeMinFreq(data, features, min_freq)
    if(normalize):
        data = normalizeData(data)

    return pd.DataFrame(data,columns = features)

In [60]:
df_bow = loadCount(lista_t, normalize = False)
#df_tfidf = loadTfidf(text_list)

In [63]:
print(df_bow['Abril'])

0     0
1     0
2     0
3     0
4     0
5     0
6     0
7     0
8     0
9     0
10    0
11    0
12    0
13    0
14    0
15    0
16    0
17    0
18    0
19    0
20    0
21    0
22    0
23    0
24    1
25    0
26    0
27    0
28    0
29    0
30    0
31    0
32    0
33    0
34    0
35    0
36    0
37    0
38    0
39    0
40    0
41    0
42    0
43    0
44    0
45    0
46    0
47    0
48    0
49    0
50    0
51    0
52    0
Name: Abril, dtype: int64


In [70]:
df['labels']

0     0
1     0
2     0
3     0
4     0
5     0
6     0
7     0
8     0
9     0
10    0
11    0
12    0
13    0
14    0
15    0
16    0
17    0
18    0
19    0
20    0
21    0
22    0
23    0
24    0
25    0
26    0
27    0
28    0
29    0
30    0
31    0
32    0
33    0
34    0
35    0
36    0
37    0
38    0
39    0
40    0
41    0
42    0
43    0
44    0
45    0
46    0
47    0
48    0
49    0
50    1
51    1
52    1
Name: labels, dtype: int64

In [89]:
feature_selection = 100

X_best = SelectKBest(mutual_info_classif,k=feature_selection).fit_transform(df_bow,df['labels'])

predictions = (cross_val_predict(model, X_best, df['labels'], cv=3)) # Will return 0 and 1s


In [90]:
#predictions = model.predict(X_best) # Will return fake and true strings

In [91]:
target_name = ['Falsas','Verdadeiras']
print(classification_report(labels, predictions, target_names=target_name))

              precision    recall  f1-score   support

      Falsas       0.94      1.00      0.97        50
 Verdadeiras       0.00      0.00      0.00         3

    accuracy                           0.94        53
   macro avg       0.47      0.50      0.49        53
weighted avg       0.89      0.94      0.92        53



C:\Users\vduchi01\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\vduchi01\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\vduchi01\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
cm = tf.math.confusion_matrix(labels=labels_2,predictions=predictions)

In [ ]:
plt.figure(figsize= (10,7))
sns.heatmap(data=cm, annot=True)
plt.xlabel('Predicted')
plt.ylabel('Truth')

Rede Neural

In [ ]:
y_pred = (model.predict(padded_t) >=0.5).astype(int)

In [ ]:
for i in range(0,50):
    print(y_pred[i])

In [ ]:
print(accuracy_score(labels, y_pred))
print(precision_score(labels, y_pred))
print(recall_score(labels, y_pred))
print(f1_score(labels, y_pred))

In [ ]:
target_name = ['Verdadeiras','Falsas']
print(classification_report(labels, y_pred, target_names=target_name))

In [ ]:
cm = tf.math.confusion_matrix(labels=labels,predictions=y_pred)

In [ ]:
plt.figure(figsize= (10,7))
sns.heatmap(data=cm, annot=True)
plt.xlabel('Predicted')
plt.ylabel('Truth')

### Classificação Fakes

In [ ]:
lista_f = []
lista_f.append(sentence_f)
lista_f

In [ ]:
maxlen = 100
sequences_f = tokenizer.texts_to_sequences(lista_f)
padded_f = pad_sequences(sequences_f, maxlen=maxlen)

In [ ]:
model.predict(padded_f)

In [ ]:
(model.predict(padded_f) >=0.5).astype(int)